In [ ]:
pip install --upgrade pip

In [ ]:
# Install requirements
!pip install -r Requirements.txt

In [ ]:
# View AWS Configuration
!aws configure list

In [ ]:
# Imports 
from ray.util import inspect_serializability
import ray
import pyarrow.fs as pq
import pandas as pd

In [ ]:
# Reusable definitions here
bucket = 'DH-SECURE-THANOS-RAY-USE'
endpoint = 'https://s3.upshift.redhat.com'

year = '2021'
month = '01'
day = '01'

# Read path
read_path = 'raydev/metric=cluster_version'
read_bucket_uri = f's3://{bucket}/{read_path}/year={year}/month={month}/day={day}'


# Write path
write_path = 'raydev-write-demo/metric=cluster_version'
write_bucket_uri = f's3://{bucket}/{write_path}/year={year}/month={month}/day={day}'


Following block reads one days worth of parquet files and writes back a single parquet file
Month 1, Day 1.
Fails currently because worker nodes keep dying.

In [ ]:
# S3FileSystem in PyArrow
fs_pyarrow = pq.S3FileSystem(endpoint_override=endpoint)

# Reading parquet using Ray through filesystem
df = ray.data.read_parquet(paths=read_bucket_uri, filesystem=fs_pyarrow)

# Writing back a single parquet file
df.repartition(1).write_parquet(path=write_bucket_uri, filesystem=fs_pyarrow)